# Automatizarea intocmirii fisei pacientului

### Scop

Dezvoltarea unui sistem inteligent care să ajute medicii pentru a completa in mod automat fisa pacientului.

### Ideea de baza

Munca medicilor este plina de provocari. Mai ales cand trebuie sa faca multe task-uri, uneori simultan, precum realizarea si citirea unei ecografii si inregistrarea observatiilor facute. De aceea este nevoie de un sistem inteligent care sa transforme informatia audio inregistrata de catre un medic in format text si sa completeze in mod automat rubricile dedicate din fisa pacientului. Se va pleca de la inregistrari audio, se vor converti in format text si se va compelta automat partea evidentiata cu galben din fisa pacientului (informatiile respective se vor salva intr-un tabel/jason si apoi se vor exporta intr-un document word)

---

### Descriere Formală (Matematică / Tehnică)

Datele de intrare:
𝑋 = { fișiere audio_i }
- Explicație: X reprezintă mulțimea fișierelor audio înregistrate de medic, fiecare corespunzând unei observații sau consultații.

Scopul aplicației:
𝐹: 𝑋 → 𝑌
- Explicație: F este funcția care transformă fișierele audio în fișe pacient completate.

Unde:
𝑌 = { fișe pacient completate }
- Explicație: Y este mulțimea fișelor completate cu informații structurate extrase din audio.

---

### Descompunerea Problemei

Problema poate fi descompusă în două sub-probleme:
##### 1. Speech-to-Text
𝑓₁ : audio → text
- Explicație: f1 folosește modele ASR (Whisper, DeepSpeech, wav2vec2 etc.) pentru a transforma fișierele audio în text.
##### 2. Information Extraction
𝑓₂ : text → date structurate
- Explicație: f2 extrage rubricile relevante din text pentru completarea automată a fișei pacientului.

### Funcția Finală

Funcția finală este:
𝐹 = 𝑓₂ ∘ 𝑓₁
- Explicație: mai întâi transformăm audio-ul în text, apoi extragem datele structurate pentru completarea fișei pacientului.



In [1]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import shutil
import os

In [2]:
app = FastAPI()
UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

In [3]:
@app.post("/upload-audio/")
async def upload_audio(file: UploadFile = File(...)):
    file_path = os.path.join(UPLOAD_FOLDER, file.filename)
    with open(file_path, "wb") as buffer:
        content = await file.read()
        buffer.write(content)
    file_size = len(content)
    response = 'TestTest'
    # response = algoritm_ML_Revolutionar(content)
    return JSONResponse({
        "filename": file.filename,
        "size_bytes": file_size,
        "message": response
    })

In [4]:
import nest_asyncio
import uvicorn
import threading

nest_asyncio.apply()

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

print("FastAPI server is running in the background on http://127.0.0.1:8000")

FastAPI server is running in the background on http://127.0.0.1:8000


INFO:     Started server process [32721]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52289 - "POST /upload-audio/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52289 - "POST /upload-audio/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52289 - "POST /upload-audio/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52289 - "POST /upload-audio/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52315 - "POST /upload-audio/ HTTP/1.1" 200 OK
